In [237]:
# Thanks to user394430 on StackOverflow for this!
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

from ipywidgets import *
from sympy import *
init_printing()
from sympy.abc import a, b, c, x, y, z
from random import *
import functools

LoL = [a, b, c, x, y, z]

degree = randint(2,4) # Randomly assign the degree of the simplified polynomial

numvars = randint(1,2) # Randomly assign the number of variables to work with

# Randomly choose between one or max elements of a list
def choose (ListOfLetters, max):
        picks = []
        while len(picks)<1 or len(picks)>max:
            picks = sample(range(0, len(ListOfLetters)), randint(1, len(ListOfLetters)))
        return [ListOfLetters[p] for p in picks]
   
# Convert input expression to Latex code
def tex(expression):
    return latex(eval(str(expression)))

# create a list of variables
varlist = choose(LoL, numvars) # Choose variables
check = 0
if len(varlist) == 1:
    check = check + 1
    varlist = varlist + [1]
    

# create a list of constants
coeffs = [x for x in range(-3,3)] if randint(1,3) in {1, 2} else [Rational(x, y) for x in {-4, -2, 0, 2, 4} for y in {-4, -2, 2, 4}] 
coefflist1 = [sample(coeffs, 2) for x in range(0,degree)]
coefflist2 = [sample(coeffs, 2) for x in range(0,degree)]
paramlist = choose(list(set(set(LoL).difference(set(varlist)))), randint(1, 2))
if len(paramlist)==1:
    paramlist = paramlist + [1]
prodlist1 = [[(x*y) for (x, y) in zip(paramlist, coefflist1[k])] for k in range(len(coefflist1))]
prodlist2 = [[(x*y) for (x, y) in zip(paramlist, coefflist2[k])] for k in range(len(coefflist2))]

# Multiply the variables by the coeffients
elements1 = [[(x*y) for (x, y) in zip(varlist, prodlist1[k])] for k in range(len(coefflist1))]
elements2 = [[(x*y) for (x, y) in zip(varlist, prodlist2[k])] for k in range(len(coefflist2))]

# Create linear factors
factors1 = [x+y for (x, y) in elements1]
factors2 = [x+y for (x, y) in elements2]
term1 = expand(prod(factors1))
term2 = expand(prod(factors2))

# Produce the problem for display
problem = tex(term1) + " + " + tex(term2) if LC(term2)>0 else tex(term1) + tex(term2)
split_problem = problem.split(" - ")
catcher = 1
while catcher !=-1:
    shuffled = (" - ").join(sample(split_problem, len(split_problem)))
    catcher = shuffled.find("- -")
    
problem = term1 + term2
    
# Obtain the degree and constant term of the expression
from sympy import degree
if check == 0: 
    v_part = Add(*[t for t in problem.args if t.has(varlist[0]) or t.has(varlist[1])]).factor(varlist[0], varlist[1])
    deg = max([(degree(t, varlist[0])+ degree(t, varlist[1])) for t in expand(v_part).args])
    c_part = Add(*[t for t in problem.args if not (t.has(varlist[0]) or t.has(varlist[1]))])
else:
    v_part = Add(*[t for t in problem.args if t.has(varlist[0])]).factor(varlist[0])
    deg = max([degree(t, varlist[0]) for t in expand(v_part).args])
    c_part = Add(*[t for t in problem.args if not (t.has(varlist[0]))])
    

solution = v_part + c_part

# Create the problem 
if check == 0:
    instructions = widgets.HTMLMath(
    value= r"(1) Treating \(" + tex(varlist) + " \) as the varibles in the expression,completely simplify:"
    )
else:
    instructions = widgets.HTMLMath(
    value=r"(1) Treating \(" + tex(varlist[0])+ "\) as the varible in the expression,completely simplify:"
    )
question = widgets.HTMLMath(
            value=r"\(" + shuffled + "\)"
)
answer = widgets.Text(
       value='0',
       placeholder="",
       description='Simplification', )
instructions2 = widgets.HTML(
    value="(2) State the degree and constant term:"
)
degree = widgets.Text(
       value='0',
       placeholder="",
       description='Degree', )
cst_term = widgets.Text(
       value='0',
       placeholder="",
       description='Constant Term', )
button = widgets.Button(description='Check Answer')
out = widgets.Output()
def on_button_clicked(_):
            with out:
                out.clear_output()
                if sympify(answer.value) - solution == 0 and sympify(degree.value) - deg == 0 and sympify(cst_term.value) - c_part == 0:
                    print("Correct!")
                else:
                    if sympify(answer.value) - solution == 0 or sympify(degree.value) - deg == 0 or sympify(cst_term.value) - c_part == 0:
                        print("Only partially correct")
                    else: print("Incorrect")
                display(widgets.HTML(
                    value="Given:"
                    ))
                display(widgets.HTMLMath(
                  value = r"\(" + shuffled +"\)"
                    ))
                display(widgets.HTML(
                    value="Gather like terms, and simplify to obtain:"
                    ))
                display(widgets.HTMLMath(
                    value=r"\(" + tex(expand(v_part)) + "\)" + "\( + " + tex(expand(c_part)) + " \)"
                    ))
                if check == 1:
                    display(widgets.HTMLMath(
                      value = r"THINK: As there is only one variable, the highest exponent of \(" + tex(varlist[0]) + "\) is the degree." 
                    ))
                    display(widgets.HTMLMath(
                      value = r"WRITE: degree = " + tex(deg) +", by inspection."
                    ))
                    display(widgets.HTMLMath(
                      value = r"THINK: The constant term is the term not being multiplied by the variable \(" + tex(varlist[0]) + "\)"
                    ))
                    display(widgets.HTMLMath(
                      value = r"WRITE: constant term = \(" + tex(c_part) + "\), by inspection."
                    ))
                else:
                    display(widgets.HTMLMath(
                      value = r"THINK: The varibles in this expression are \(" + tex(varlist[0]) + "\) and \(" + tex(varlist[1]) + "\). The highest of their exponents independently, or their sum in a product, is the degree. " 
                    ))
                    display(widgets.HTMLMath(
                      value = r"WRITE: degree = " + tex(deg) +", by inspection."
                    ))
                    display(widgets.HTMLMath(
                      value = r"THINK: The constant term is the term not being multiplied by either variable \(" + tex(varlist[0]) + "\) or \(" + tex(varlist[1]) +"\)."
                    ))
                    display(widgets.HTMLMath(
                      value = r"WRITE: constant term = \(" + tex(c_part) + "\), by inspection."
                    ))
button.on_click(on_button_clicked)
widgets.VBox([instructions, question, answer, instructions2, degree, cst_term, button, out])

Toggle code

In [232]:
problem

In [233]:
term1

In [234]:
term2

In [239]:
X = [a, 2 * b]
[x for x in X]

In [241]:
Y = [-3 * a, b**2, c]
[x*y for x in X for y in Y]

In [249]:
A=[x for x in range(1,21+1) if 21%x == 0] + [-x for x in range(1,21+1) if 21%x == 0]

In [256]:
A2=[(x,-21/x) for x in A ]

In [253]:
B = [(x, 2/x) for x in range(1,2+1) if 2%x == 0] + [(-x, 2/-x) for x in range(1,2+1) if 2%x == 0]
B

In [259]:
[(a*b) for (a,b) in zip(A2, B) ]

TypeError: can't multiply sequence by non-int of type 'tuple'

In [260]:
48/6